# Internship - Public Data Internship sponsored by Korean government and ITX Hyosung

## 1. Excel process of pivot table
    : Descriptions about how to improve the efficiency. 

In [24]:
import pandas as pd
import numpy as np
import dataframe as df
import openpyxl
from pandas import Series, DataFrame

In [25]:
df1 =pd.read_csv('sheet8.csv', engine = 'python')
#  loaded file needs pivot tabling. 

In [26]:
df1 = pd.read_csv('sheet8.csv', engine = 'python', thousands = ',')
#  Reload the file with regards the thousands mark in read_csv
# Below are the previous one that handles thousands mark after it is loaded. 

# for i in range(len(df)):
#     if ',' in df['전체건수'][i]:
#         df['전체건수'][i]=df['전체건수'][i].split(',')
#         df['전체건수'][i]="".join(df['전체건수'][i])      
#     else:
#         pass

# for i in range(len(df)):
#     if ',' in df['오류건수'][i]:
#         df['오류건수'][i]=df['오류건수'][i].split(',')
#         df['오류건수'][i]="".join(df['오류건수'][i])      
#     else:
#         pass

df=df1[['테이블', '도메인', '전체건수', '오류건수']]
df['오류건수']=pd.to_numeric(df['오류건수'])    
df['전체건수']=pd.to_numeric(df['전체건수'])
df

<ipython-input-26-97821feb8e15>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['오류건수']=pd.to_numeric(df['오류건수'])
<ipython-input-26-97821feb8e15>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['전체건수']=pd.to_numeric(df['전체건수'])


,테이블,도메인,전체건수,오류건수
0,LTIS.LACQ011,여부,1,0
1,LTIS.LACQ029,여부,0,0
2,LTIS.LACQ030,여부,306623,0
3,LTIS.LACQ030,여부,2560,0
4,LTIS.LACQ030,여부,5784,0
...,...,...,...,...
2903,LTIS.LRLT141,코드,10812,0
2904,LTIS.LRLT144,코드,212364,6
2905,LTIS.LRLT149,코드,96,0
2906,LTIS.SCOD147,코드,12,0


In [27]:
# Next step is to find the error rate which can be driven by number of error data / the number of total data 


# Multi Indexing required. 
# First, it needs to be ordered under the name of '테이블'
# Next, ordered under the name of ' 도메인'


## Groupby 
## Access to a column and its sum can be achieved by grouping the indexes from the base level to the highest level.
## Syntax
## Multi indexed table driven=df.groupby(['first_level_index', 'second_level_index', ...])


total = df.groupby(['테이블', '도메인'])['전체건수'].sum()
totals=pd.DataFrame(total)
error = df.groupby(['테이블', '도메인'])['오류건수'].sum()
errors = pd.DataFrame(error)
newdf = pd.concat([totals, errors], axis = 1)
newdf
newdf['오류율']=newdf['오류건수']/newdf['전체건수']
newdf

전체건수  오류건수  오류율
테이블          도메인                   
LTIS.LACQ004 날짜        0     0  NaN
             번호        0     0  NaN
             율         0     0  NaN
LTIS.LACQ006 금액   348274     0  0.0
             날짜   348274     0  0.0
...                  ...   ...  ...
LTIS.TDEP020 금액        0     0  NaN
             날짜        0     0  NaN
             번호        0     0  NaN
             수량        0     0  NaN
             여부        0     0  NaN

[755 rows x 3 columns]

In [79]:
# A template is given from my boss, which has structured form of table. 
# The groupby dataframe cannot be used in itsef so needs to be transformed to fit in the template. 
# newdf is useless for now
# The template has index of '테이블', column of ' 도메인', values of the sum of '전체건수', '오류건수', '오류율' which are total data, errored data, error rate of the data


## pivot_table(values = , index = , columns= , aggfunc= )

# Make first dataframe for all the data ' 전체건수' with values of its sum. 
pivot_all= df.pivot_table(values = '전체건수', index = '테이블', columns= '도메인', aggfunc= sum)

#check if all the columns in the template is in the pivot_all dataframe. if not, we need to add one more columnn at the end of pivot_all. 
# print(pivot_all)

#rename the columns following the template. 
pivot_all= pivot_all [['날짜', '여부', '번호','금액', '수량', '율', '코드']]

# Make second dataframe and printing, renaming it again. 
pivot_e= df.pivot_table(values = '오류건수', index = '테이블', columns= '도메인', aggfunc= sum)
# print(pivot_e)
pivot_e= pivot_e [['날짜', '여부', '번호','금액', '수량', '율', '코드']]


#Previously, I changed the column name of pivot_e to make renaming process following easy. 
# pivot_e.columns = ['날짜1', '여부1', '번호1', '금액1', '수량1', '율1', '코드1'] 
#pivot = pd.concat([pivot_all, pivot_e], axis = 1)
# pivot = pivot[['날짜', '날짜1', '여부', '여부1', '번호', '번호1', '금액', '금액1', '수량', '수량1', '율', '율1', '코드', '코드1']]
# pivot['날짜오류'] = pivot.iloc[:,1]/pivot.iloc[:,0]
# pivot['여부오류'] = pivot.iloc[:,3]/pivot.iloc[:,2]
# pivot['번호오류'] = pivot.iloc[:,5]/pivot.iloc[:,4]
# pivot['금액오류'] = pivot.iloc[:,7]/pivot.iloc[:,6]
# pivot['수량오류'] = pivot.iloc[:,9]/pivot.iloc[:,8]
# pivot['율오류'] = pivot.iloc[:,11]/pivot.iloc[:,10]
# pivot['코드오류'] = pivot.iloc[:,13]/pivot.iloc[:,12]
# pivot=pivot[['날짜', '날짜1', '날짜오류', '여부', '여부1', '여부오류', '번호', '번호1', '번호오류', '금액', '금액1', '금액오류', '수량', '수량1', '수량오류', '율', '율1', '율오류','코드', '코드1', '코드오류']]
#pivot.to_csv('Complete_sheet9.csv')


#Instead, Multi Indexing is used and the logic is given by
# https://qastack.kr/programming/58802654/merge-two-dataframes-and-add-column-level-with-names
# First make indexes that will be used for the multiindex dataframe
# In the columnss, from_product is used that 1) uppoer level column names, and a list of the lower level column names required in the templates. 

columnss = pd.MultiIndex.from_product([pivot_all.columns.values, ['전체건수', '오류건수', '오류율']])

# Make an empty dataframe to fill it up from following codes. 
empty_df = pd.DataFrame('-', index = pivot_all.index, columns = columnss)

# Now fill the dataframe using the column names of the previous flatten two dataframe, pivot_all and pivot_e. 
for c in pivot_all.columns: 
    empty_df.loc[:, (c, '전체건수')] = pivot_all[c]
for co in pivot_e.columns: 
    empty_df.loc[:, (co, '오류건수')] = pivot_e[co]

# Calculate between the lower leveled columns in the empty_df using the access syntax for multiindex. 
for rate in empty_df.columns.levels[0]:
    empty_df[rate, '오류율'] = empty_df[rate, '오류건수']/empty_df[rate, '전체건수']
empty_df

                    날짜              여부                 번호             \
                  전체건수 오류건수 오류율   전체건수 오류건수 오류율      전체건수   오류건수 오류율   
테이블                                                                    
LTIS.LACQ004       0.0  0.0   -    NaN  NaN   -       0.0    0.0   -   
LTIS.LACQ006  348274.0  0.0   -    NaN  NaN   -  348274.0    0.0   -   
LTIS.LACQ009       NaN  NaN   -    NaN  NaN   -       3.0    0.0   -   
LTIS.LACQ011       NaN  NaN   -    1.0  0.0   -      47.0    0.0   -   
LTIS.LACQ024       NaN  NaN   -    NaN  NaN   -     615.0  272.0   -   
...                ...  ...  ..    ...  ...  ..       ...    ...  ..   
LTIS.SCOD255       8.0  0.0   -    NaN  NaN   -      16.0    0.0   -   
LTIS.SCOD290       NaN  NaN   -    9.0  0.0   -       NaN    NaN   -   
LTIS.SCOD334       NaN  NaN   -    9.0  0.0   -       9.0    9.0   -   
LTIS.SCOD380       NaN  NaN   -  141.0  0.0   -       NaN    NaN   -   
LTIS.TDEP020       0.0  0.0   -    0.0  0.0   -       0.0    0.0

날짜               여부                  번호                   \
                  전체건수 오류건수  오류율   전체건수 오류건수  오류율      전체건수   오류건수       오류율   
테이블                                                                            
LTIS.LACQ004       0.0  0.0  NaN    NaN  NaN  NaN       0.0    0.0       NaN   
LTIS.LACQ006  348274.0  0.0  0.0    NaN  NaN  NaN  348274.0    0.0  0.000000   
LTIS.LACQ009       NaN  NaN  NaN    NaN  NaN  NaN       3.0    0.0  0.000000   
LTIS.LACQ011       NaN  NaN  NaN    1.0  0.0  0.0      47.0    0.0  0.000000   
LTIS.LACQ024       NaN  NaN  NaN    NaN  NaN  NaN     615.0  272.0  0.442276   
...                ...  ...  ...    ...  ...  ...       ...    ...       ...   
LTIS.SCOD255       8.0  0.0  0.0    NaN  NaN  NaN      16.0    0.0  0.000000   
LTIS.SCOD290       NaN  NaN  NaN    9.0  0.0  0.0       NaN    NaN       NaN   
LTIS.SCOD334       NaN  NaN  NaN    9.0  0.0  0.0       9.0    9.0  1.000000   
LTIS.SCOD380       NaN  NaN  NaN  141.0  0.0  0.0       NaN    NaN       NaN   
LTIS.TDEP020       0.0  0.0  NaN    0.0  0.0  NaN       0.0    0.0       NaN   

                    금액  ...             수량                   율            \
                  전체건수  ...  오류율      전체건수 오류건수  오류율      전체건수 오류건수  오류율   
테이블                     ...                                                
LTIS.LACQ004       NaN  ...  NaN       NaN  NaN  NaN       0.0  0.0  NaN   
LTIS.LACQ006  348274.0  ...  0.0  348274.0  0.0  0.0  665660.0  0.0  0.0   
LTIS.LACQ009       NaN  ...  NaN       NaN  NaN  NaN       NaN  NaN  NaN   
LTIS.LACQ011       NaN  ...  NaN       NaN  NaN  NaN       NaN  NaN  NaN   
LTIS.LACQ024       NaN  ...  NaN       NaN  NaN  NaN       NaN  NaN  NaN   
...                ...  ...  ...       ...  ...  ...       ...  ...  ...   
LTIS.SCOD255       NaN  ...  NaN       NaN  NaN  NaN       8.0  0.0  0.0   
LTIS.SCOD290       NaN  ...  NaN       NaN  NaN  NaN       NaN  NaN  NaN   
LTIS.SCOD334       NaN  ...  NaN       NaN  NaN  NaN       NaN  NaN  NaN   
LTIS.SCOD380       NaN  ...  NaN       NaN  NaN  NaN       NaN  NaN  NaN   
LTIS.TDEP020       0.0  ...  NaN       0.0  0.0  NaN       NaN  NaN  NaN   

                    코드            
                  전체건수 오류건수  오류율  
테이블                               
LTIS.LACQ004       NaN  NaN  NaN  
LTIS.LACQ006  348274.0  0.0  0.0  
LTIS.LACQ009       NaN  NaN  NaN  
LTIS.LACQ011       NaN  NaN  NaN  
LTIS.LACQ024       NaN  NaN  NaN  
...                ...  ...  ...  
LTIS.SCOD255       NaN  NaN  NaN  
LTIS.SCOD290       NaN  NaN  NaN  
LTIS.SCOD334       NaN  NaN  NaN  
LTIS.SCOD380       NaN  NaN  NaN  
LTIS.TDEP020       NaN  NaN  NaN  

[210 rows x 21 columns]

In [70]:
empty_df.to_csv('sheetttttt.csv')

# The codes


import pandas as pd
import numpy as np
import dataframe as df
import openpyxl
from pandas import Series, DataFrame

df1 = pd.read_csv('sheet8.csv', engine = 'python', thousands = ',')

df=df1[['테이블', '도메인', '전체건수', '오류건수']]
df['오류건수']=pd.to_numeric(df['오류건수'])    
df['전체건수']=pd.to_numeric(df['전체건수'])
df

pivot_all= df.pivot_table(values = '전체건수', index = '테이블', columns= '도메인', aggfunc= sum)
pivot_all= pivot_all [['날짜', '여부', '번호','금액', '수량', '율', '코드']]

pivot_e= df.pivot_table(values = '오류건수', index = '테이블', columns= '도메인', aggfunc= sum)
pivot_e= pivot_e [['날짜', '여부', '번호','금액', '수량', '율', '코드']]

columnss = pd.MultiIndex.from_product([pivot_all.columns.values, ['전체건수', '오류건수', '오류율']])
empty_df = pd.DataFrame('-', index = pivot_all.index, columns = columnss)

for c in pivot_all.columns: 
    empty_df.loc[:, (c, '전체건수')] = pivot_all[c]
for co in pivot_e.columns: 
    empty_df.loc[:, (co, '오류건수')] = pivot_e[co]
for rate in empty_df.columns.levels[0]:
    empty_df[rate, '오류율'] = empty_df[rate, '오류건수']/empty_df[rate, '전체건수']

empty_df